# GPU Patent Data Pipeline v1: Core, Expansion, and ExpansionXVocab Datasets

This notebook implements a three-dataset pipeline for patent analysis:

## Dataset Definitions

1. **Core Dataset**: Patents with CPC codes related to:
   - G06F 9/3887, G06F 9/3888, G06F 9/38885 (Parallel processing)
   - G06F 9/3009 (Multiprocessing arrangements)
   - G06F 12/0842, G06F 12/0844 (Cache memory)
   - G06F 13/42, G06F 13/14, G06F 13/16 (Bus architectures)

2. **Expansion Dataset**: Patents with CPC codes:
   - G06F 15/8007, G06F 15/8053 (Multiprocessor systems)
   - G06N 3/06 (Neural networks)

3. **ExpansionXVocab Dataset**: Patents from Expansion that contain keywords:
   - gpu
   - high-performance compute
   - hpc

## Setup
1. Install required packages: `pip install requests pandas pyarrow`
2. Set your lens.org API token in environment variable `LENS_API_TOKEN`

In [ ]:
import os
import json
import gzip
import re
import requests
import pandas as pd
from datetime import datetime
from pathlib import Path

## Configuration

In [ ]:
# API Configuration
API_TOKEN = os.getenv('LENS_API_TOKEN', 'your-api-token-here')
API_URL = 'https://api.lens.org/patent/search'

# Base data path
BASE_PATH = Path('../data/patents/v1_core_expansion')

# Dataset-specific CPC codes
CORE_CPC_CODES = [
    'G06F9/3887',   # Parallel processing
    'G06F9/3888',   # Parallel processing
    'G06F9/38885',  # Parallel processing
    'G06F9/3009',   # Multiprocessing arrangements
    'G06F12/0842',  # Cache memory
    'G06F12/0844',  # Cache memory
    'G06F13/42',    # Bus architectures
    'G06F13/14',    # Bus architectures
    'G06F13/16',    # Bus architectures
]

EXPANSION_CPC_CODES = [
    'G06F15/8007',  # Multiprocessor systems
    'G06F15/8053',  # Multiprocessor systems
    'G06N3/06',     # Neural networks
]

# Keywords for ExpansionXVocab filtering
VOCAB_KEYWORDS = [
    'gpu',
    'high-performance compute',
    'hpc'
]

# Search parameters
JURISDICTION = 'US'  # United States patents only
MAX_RESULTS = 1000   # Max results per query (adjust based on needs)

## Helper Functions

In [ ]:
def create_directories(base_path, dataset_name):
    """Create directory structure for a dataset."""
    dataset_path = base_path / dataset_name
    for subdir in ['raw', 'parsed', 'text_clean', 'embeddings', 'logs']:
        (dataset_path / subdir).mkdir(parents=True, exist_ok=True)
    return dataset_path

def build_cpc_query(cpc_codes, jurisdiction, max_results=1000):
    """
    Build lens.org API query for patents with specific CPC codes.
    
    Args:
        cpc_codes: List of CPC classification codes
        jurisdiction: Patent jurisdiction (e.g., 'US')
        max_results: Maximum number of results to return
    
    Returns:
        Query dictionary for lens.org API
    """
    query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"term": {"classification_cpc.classification_id": code}} 
                                for code in cpc_codes
                            ],
                            "minimum_should_match": 1
                        }
                    },
                    {"term": {"jurisdiction": jurisdiction}}
                ]
            }
        },
        "size": max_results,
        "include": [
            "lens_id",
            "title",
            "abstract",
            "description",
            "claims",
            "date_published",
            "jurisdiction",
            "applicants",
            "inventors",
            "classification_cpc",
            "biblio"
        ]
    }
    return query

def fetch_patents(api_token, query):
    """
    Fetch patents from lens.org API.
    
    Args:
        api_token: lens.org API token
        query: Query dictionary
    
    Returns:
        API response data
    """
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Content-Type': 'application/json'
    }
    
    response = requests.post(API_URL, json=query, headers=headers)
    response.raise_for_status()
    
    return response.json()

def save_raw_data(raw_data, output_path, dataset_name, timestamp):
    """Save raw data as compressed JSON."""
    raw_file = output_path / 'raw' / f'{dataset_name}_{timestamp}.json.gz'
    with gzip.open(raw_file, 'wt', encoding='utf-8') as f:
        json.dump(raw_data, f, indent=2)
    return raw_file

def parse_patents(raw_data):
    """
    Parse raw patent data into normalized schema.
    
    Args:
        raw_data: Raw API response
    
    Returns:
        DataFrame with normalized patent data
    """
    patents = raw_data.get('data', [])
    
    parsed_records = []
    for patent in patents:
        record = {
            'lens_id': patent.get('lens_id'),
            'title': patent.get('title', ''),
            'abstract': patent.get('abstract', ''),
            'description': patent.get('description', ''),
            'date_published': patent.get('date_published'),
            'jurisdiction': patent.get('jurisdiction'),
            'applicants': json.dumps(patent.get('applicants', [])),
            'inventors': json.dumps(patent.get('inventors', [])),
            'cpc_codes': json.dumps([c.get('classification_id') for c in patent.get('classification_cpc', [])]),
            'claims_count': len(patent.get('claims', [])),
            'first_claim': patent.get('claims', [{}])[0].get('claim_text') if patent.get('claims') else None
        }
        parsed_records.append(record)
    
    return pd.DataFrame(parsed_records)

def save_parsed_data(df, output_path, dataset_name, timestamp):
    """Save parsed data as parquet."""
    parsed_file = output_path / 'parsed' / f'{dataset_name}_{timestamp}.parquet'
    df.to_parquet(parsed_file, index=False)
    return parsed_file

def contains_keywords(text, keywords):
    """
    Check if text contains any of the specified keywords (case-insensitive).
    
    Args:
        text: Text to search in
        keywords: List of keywords to search for
    
    Returns:
        True if any keyword is found, False otherwise
    """
    if not text or pd.isna(text):
        return False
    
    text_lower = text.lower()
    return any(keyword.lower() in text_lower for keyword in keywords)

def filter_by_keywords(df, keywords):
    """
    Filter dataframe to include only patents containing specified keywords.
    Searches in title, abstract, and description fields.
    
    Args:
        df: DataFrame with patent data
        keywords: List of keywords to filter by
    
    Returns:
        Filtered DataFrame
    """
    mask = df.apply(
        lambda row: (
            contains_keywords(row['title'], keywords) or
            contains_keywords(row['abstract'], keywords) or
            contains_keywords(row['description'], keywords)
        ),
        axis=1
    )
    return df[mask].copy()

def log_pipeline_run(output_path, dataset_name, timestamp, query, results_count, file_info):
    """Log pipeline execution details."""
    log_file = output_path / 'logs' / f'{dataset_name}_{timestamp}.log.json'
    log_data = {
        'timestamp': timestamp,
        'dataset': dataset_name,
        'query': query,
        'results_count': results_count,
        'files': file_info
    }
    with open(log_file, 'w') as f:
        json.dump(log_data, f, indent=2)
    return log_file

## 1. Fetch and Process Core Dataset

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

print("="*80)
print("CORE DATASET PIPELINE")
print("="*80)

# Create directories
core_path = create_directories(BASE_PATH, 'core')
print(f"\n✓ Directories created at: {core_path}")

# Build query
core_query = build_cpc_query(CORE_CPC_CODES, JURISDICTION, MAX_RESULTS)
print(f"\n📋 Query configuration:")
print(f"   CPC Codes: {len(CORE_CPC_CODES)} codes")
print(f"   Jurisdiction: {JURISDICTION}")
print(f"   Max Results: {MAX_RESULTS}")

# Fetch data
print(f"\n🔍 Fetching core patents from lens.org...")
try:
    core_raw_data = fetch_patents(API_TOKEN, core_query)
    core_count = len(core_raw_data.get('data', []))
    print(f"✓ Successfully fetched {core_count} patents")
    print(f"  Total results available: {core_raw_data.get('total', 0)}")
    
    # Save raw data
    core_raw_file = save_raw_data(core_raw_data, core_path, 'core', timestamp)
    print(f"\n✓ Raw data saved to: {core_raw_file}")
    print(f"  File size: {core_raw_file.stat().st_size / 1024:.2f} KB")
    
    # Parse and save
    df_core = parse_patents(core_raw_data)
    core_parsed_file = save_parsed_data(df_core, core_path, 'core', timestamp)
    print(f"\n✓ Parsed data saved to: {core_parsed_file}")
    print(f"  Records: {len(df_core)}")
    
    # Log execution
    core_log = log_pipeline_run(
        core_path, 'core', timestamp, core_query, core_count,
        {'raw': str(core_raw_file), 'parsed': str(core_parsed_file)}
    )
    print(f"\n✓ Log saved to: {core_log}")
    
    print(f"\n📊 Sample record:")
    if len(df_core) > 0:
        print(df_core[['lens_id', 'title']].head(3).to_string(index=False))
    
except Exception as e:
    print(f"✗ Error: {e}")
    core_raw_data = None
    df_core = None

## 2. Fetch and Process Expansion Dataset

In [ ]:
print("\n" + "="*80)
print("EXPANSION DATASET PIPELINE")
print("="*80)

# Create directories
expansion_path = create_directories(BASE_PATH, 'expansion')
print(f"\n✓ Directories created at: {expansion_path}")

# Build query
expansion_query = build_cpc_query(EXPANSION_CPC_CODES, JURISDICTION, MAX_RESULTS)
print(f"\n📋 Query configuration:")
print(f"   CPC Codes: {len(EXPANSION_CPC_CODES)} codes")
print(f"   Jurisdiction: {JURISDICTION}")
print(f"   Max Results: {MAX_RESULTS}")

# Fetch data
print(f"\n🔍 Fetching expansion patents from lens.org...")
try:
    expansion_raw_data = fetch_patents(API_TOKEN, expansion_query)
    expansion_count = len(expansion_raw_data.get('data', []))
    print(f"✓ Successfully fetched {expansion_count} patents")
    print(f"  Total results available: {expansion_raw_data.get('total', 0)}")
    
    # Save raw data
    expansion_raw_file = save_raw_data(expansion_raw_data, expansion_path, 'expansion', timestamp)
    print(f"\n✓ Raw data saved to: {expansion_raw_file}")
    print(f"  File size: {expansion_raw_file.stat().st_size / 1024:.2f} KB")
    
    # Parse and save
    df_expansion = parse_patents(expansion_raw_data)
    expansion_parsed_file = save_parsed_data(df_expansion, expansion_path, 'expansion', timestamp)
    print(f"\n✓ Parsed data saved to: {expansion_parsed_file}")
    print(f"  Records: {len(df_expansion)}")
    
    # Log execution
    expansion_log = log_pipeline_run(
        expansion_path, 'expansion', timestamp, expansion_query, expansion_count,
        {'raw': str(expansion_raw_file), 'parsed': str(expansion_parsed_file)}
    )
    print(f"\n✓ Log saved to: {expansion_log}")
    
    print(f"\n📊 Sample record:")
    if len(df_expansion) > 0:
        print(df_expansion[['lens_id', 'title']].head(3).to_string(index=False))
    
except Exception as e:
    print(f"✗ Error: {e}")
    expansion_raw_data = None
    df_expansion = None

## 3. Filter and Process ExpansionXVocab Dataset

In [ ]:
print("\n" + "="*80)
print("EXPANSIONXVOCAB DATASET PIPELINE")
print("="*80)

if df_expansion is not None:
    # Create directories
    expansionxvocab_path = create_directories(BASE_PATH, 'expansionxvocab')
    print(f"\n✓ Directories created at: {expansionxvocab_path}")
    
    print(f"\n📋 Filter configuration:")
    print(f"   Source: Expansion dataset ({len(df_expansion)} patents)")
    print(f"   Keywords: {', '.join(VOCAB_KEYWORDS)}")
    
    # Filter by keywords
    print(f"\n🔍 Filtering expansion patents by keywords...")
    df_expansionxvocab = filter_by_keywords(df_expansion, VOCAB_KEYWORDS)
    vocab_count = len(df_expansionxvocab)
    print(f"✓ Filtered to {vocab_count} patents ({vocab_count/len(df_expansion)*100:.1f}% of expansion)")
    
    # Save filtered data (we use the expansion raw data as source)
    if vocab_count > 0:
        # Create a filtered raw dataset
        filtered_raw_data = {
            'total': vocab_count,
            'data': [
                patent for patent in expansion_raw_data.get('data', [])
                if patent.get('lens_id') in df_expansionxvocab['lens_id'].values
            ]
        }
        
        vocab_raw_file = save_raw_data(filtered_raw_data, expansionxvocab_path, 'expansionxvocab', timestamp)
        print(f"\n✓ Raw data saved to: {vocab_raw_file}")
        print(f"  File size: {vocab_raw_file.stat().st_size / 1024:.2f} KB")
        
        # Save parsed data
        vocab_parsed_file = save_parsed_data(df_expansionxvocab, expansionxvocab_path, 'expansionxvocab', timestamp)
        print(f"\n✓ Parsed data saved to: {vocab_parsed_file}")
        print(f"  Records: {len(df_expansionxvocab)}")
        
        # Log execution
        vocab_log = log_pipeline_run(
            expansionxvocab_path, 'expansionxvocab', timestamp,
            {'source': 'expansion', 'keywords': VOCAB_KEYWORDS}, vocab_count,
            {'raw': str(vocab_raw_file), 'parsed': str(vocab_parsed_file)}
        )
        print(f"\n✓ Log saved to: {vocab_log}")
        
        print(f"\n📊 Sample record:")
        print(df_expansionxvocab[['lens_id', 'title']].head(3).to_string(index=False))
    else:
        print("\n⚠ No patents matched the keyword filter")
else:
    print("\n✗ Cannot process ExpansionXVocab: Expansion dataset not available")

## 4. Summary Statistics

In [ ]:
print("\n" + "="*80)
print("PIPELINE SUMMARY")
print("="*80)

print(f"\nTimestamp: {timestamp}")
print(f"\nDataset Statistics:")
print(f"  Core Dataset:          {len(df_core) if df_core is not None else 0:5d} patents")
print(f"  Expansion Dataset:     {len(df_expansion) if df_expansion is not None else 0:5d} patents")
print(f"  ExpansionXVocab:       {len(df_expansionxvocab) if 'df_expansionxvocab' in locals() and df_expansionxvocab is not None else 0:5d} patents")

print(f"\nCPC Code Coverage:")
print(f"  Core CPC codes:        {len(CORE_CPC_CODES)}")
print(f"  Expansion CPC codes:   {len(EXPANSION_CPC_CODES)}")

print(f"\nKeyword Filters:")
print(f"  Keywords used:         {', '.join(VOCAB_KEYWORDS)}")

print(f"\n✓ Pipeline execution complete!")
print(f"\nAll data saved to: {BASE_PATH}")